In [ ]:
#custom prompt for this tool: RETURNS ONLY THE COMMAND
def Tool1(history):
    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents="\n".join([str(h) for h in history])
    )
    #history.append({"role": "model", "text": [f"Command generated by the model:\n{response.text}"]})
    
    return response.text

#not the same prompt as tool1 but take the command provided by tool1
def tool2(history, command):
    with open('output_cmd/output.txt', 'r') as f:
        output = f.read()
    history.append({"previous command": [command], "command output": [output]})

    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents="\n".join([str(h) for h in history])
    )
    return response.text


In [ ]:
from Prompts import prompter
i = -1
nb_questions = len(questions)
history = []
final_answers= ["Answer not found"] * nb_questions  # Initialize with default answer
previous_input = None
output1 = None
# Main control loop
while i < nb_questions:
    user_input = input("\nChoose:\n1. Next Question\n2. Analyze/Continue\n3. Exit\n>>> ")
    if previous_input is None and user_input not in ["1", "3"]:
        print("❌ You must start with option 1.")
        continue

    
    if user_input == previous_input and user_input == "1":
        print("❌ You already chose 1. You must alternate between 1 and 2.")
        continue

    previous_input = user_input  # Store the last input for validation

    if user_input == "1":
        if i == nb_questions - 1:
            print("✅ Task finished.")
            break
        i += 1
        prompt = prompter.prompt_tool1_thought(i, scenario, tools, file, questions)    #first prompt for tool1 thought
        history = [{"role": "system", "text": [prompt]}]  # start fresh for new question
        output0 = Tool1(history)
        print(f"Thought generated for question {i+1} by tool 1: {output1}")
        prompt = prompter.prompt_tool1_action(i, scenario, tools, file, questions)    #first prompt for tool1 
        history = [{"role": "system", "text": [prompt]},{"Thought"}]  # start fresh for new question
        output1 = Tool1(history)
        print(f"Command generated for question {i+1} by tool 1: {output1}")
        ExecuteCmd(output1)  # Execute the command generated by Tool1
        #we should modify the history here for the tool2: new history becomes empty -> receives the new prompt -> receives command
        prompt = prompter.prompt_tool2_zero_shot(i, scenario, tools, file, questions)    #first prompt for tool2
        history=[{"role": "system", "text": [prompt]}]  # start fresh for new question

    elif user_input == "2":
        output1 = tool2(history, output1)
        print(history)
        if "final answer:" in output1.lower():
            final_answers[i] = output1.lower().split("final answer:")[-1].strip()
            print(f"Answer to the question {i+1}: {final_answers[i]}, click 1 to move to next question")
            
        else:
            print(f"New command: {output1}, press 2 to execute it ")
            ExecuteCmd(output1)
        
        #if final result append to list else exucute cmd
    elif user_input == "3":
        print("⛔ Stopped by user.")
        break

    else:
        print("⚠️ Invalid input. Please enter 1, 2, or 3.")
